In [27]:
from ultralytics import YOLO, NAS
import sys # to access the system
import cv2
from bs4 import BeautifulSoup
import bs4
import time
import os
import subprocess
import torch
import super_gradients
import supervision

In [28]:
def path2img(img_num, dbpath):
    img_pad = str(img_num).zfill(6)
    return f"{dbpath}\\frame_{img_pad}.PNG"

def load_annotations(annotation_path: str):
    """
    annotation_path: str = r"D:\Dataset\FishDetection\Annotation\task_fish_detection-2024_02_05_18_52_05-cvat for video 1.1"
    Give full path to the CVAT formatted folder which contains annotations.xml
    """
    path=f"{annotation_path}\\annotations.xml"
    with open(path, 'r') as f:
        data = f.read()
    Bs_data = BeautifulSoup(data, "xml")
    metadata = Bs_data.find('meta').find("task").find("segments")
    # Get the annotations frame count
    start_frame = int(metadata.find("start").text)
    stop_frame = int(metadata.find("stop").text)
    # The annotations is within track property
    box_annotations = Bs_data.find_all('track')
    # Check if there are any annotations of fish, else return empty box annotations
    if box_annotations:
        # Only get box elements
        map_frame_to_index = {}
        all_boxes = box_annotations[0].find_all('box')
        for i in range(1, len(box_annotations)):
            all_boxes += box_annotations[i].find_all('box')
        for j, box in enumerate(all_boxes):
            #print(str(box["frame"]))
            if box["frame"] in map_frame_to_index:
                map_frame_to_index[box["frame"]] += [j]
            else:
                map_frame_to_index[box["frame"]] = [j]
        return all_boxes, map_frame_to_index, (start_frame, stop_frame)
    return "", {}, (start_frame, stop_frame)

def convert_png_to_video(input_path: str, output_path: str, fps: int = 30, crf: int = 23):
    """
    Convert a sequence of PNG files to a video using ffmpeg.
    # Example usage:
    # convert_png_to_video(
    #     input_path="path/to/png/folder",
    #     output_path="output_video.mp4",
    #     fps=30,
    #     crf=23
    # )
    
    Args:
        input_path: Path to folder containing PNG files (should end with %d.png format)
        output_path: Output video file path (.mp4)
        fps: Frames per second (default: 30)
        crf: Constant Rate Factor - video quality (18-28, lower is better quality)
    """
    ffmpeg_cmd = [
        'ffmpeg',
        '-framerate', str(fps),
        '-i', os.path.join(input_path, 'frame_%06d.PNG'),  # Assumes files are named 1.png, 2.png, etc.
        '-c:v', 'libx264',
        '-crf', str(crf),
        '-pix_fmt', 'yuv420p',
        output_path
    ]
    
    try:
        subprocess.run(ffmpeg_cmd, check=True)
        print(f"Video created successfully: {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error creating video: {e}")




#img = cv2.imread("sheep.png", cv2.IMREAD_ANYCOLOR)
DBPATH = r"D:\Dataset\FishDetection\Annotation\task_fish_detection-2024_01_28_18_07_53-cvat for video 1.1\images"
ANNOTATIONS= r"D:\Dataset\FishDetection\Annotation\task_fish_detection-2024_01_28_18_07_53-cvat for video 1.1"
DBROOT= r"D:\Dataset\FishDetection\Annotation"

In [110]:
torch.cuda.is_available()

True

In [76]:
# Load a COCO-pretrained YOLO11n model
model = YOLO("yolo11n.pt")
# Check https://docs.ultralytics.com/models/ for other models

# Example: Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data="coco8.yaml", epochs=100, imgsz=640)

In [78]:
directories = os.listdir(DBROOT)
for i in range(5, 20, 1):
    convert_png_to_video(f"{DBROOT}\{directories[1]}\images", f"test{i}.mp4")
        #print(f"{DBROOT}\{directories[i]}")
        #continueNext = showVideo(f"{DBROOT}\{directories[i]}",windowID, skip_empty=True)
    #p = path2img(i, f"{DBROOT}\{directories[i]}"+"\images")
    #print(p)



In [ ]:
convert_png_to_video(r"D:\Dataset\FishDetection\Annotation\task_20230716080335_20230716080344_3.mp4_dataset_2024_02_26_06_54_04_cvat for video 1.1\images", "fish.mp4")

In [155]:
# Validation
#rootpath = r"D:\Dataset\FishDetection2Yolo\data.yaml"
rootpath = r"C:\Users\alf\Documents\programming\python\IMT4392\Project\datasets\fish.v1-fish_imagesize_320.yolov11\data.yaml"
#FishDetection\Annotation"
model = YOLO(r"./runs/detect/train15/weights/best.pt")
metrics = model.val(data=rootpath,save_json=True)

val: Scanning C:\Users\alf\Documents\programming\python\IMT4392\Project\datasets\fish.v1-fish_imagesize_320.yolov11\val
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06


0.17435146167556403

In [156]:
display(metrics.box.map)  # mAP50-95
display(metrics.box.map50)  # mAP50
display(metrics.box.map75)  # mAP75
display(metrics.box.maps)

0.17435146167556403

0.33502453644448804

0.16982630220761663

array([    0.17435])

In [ ]:
results = model(r"C:\Users\alf\Videos\WokeTutorial.mp4",save=True, show=False)

In [ ]:
#model = YOLO("yolov10n.pt")
model = YOLO("yolo11n.pt")
results = model("fish.mp4",save=True, show=False)